In [5]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

## MCAD Dataset Features

The MCAD dataset contains network flow data with the following features:

**IP Address Information:**

* `src`: Source IP address
* `dst`: Destination IP address

**Flow Information:**

* `table_id`: ID of the table where the flow is stored
* `ip_bytes`: Number of bytes in the flow on the IP address level
* `ip_packet`: Number of packets in the flow on the IP address level
* `ip_duration`: Duration (in seconds) the flow has been alive on the IP address level

**Port Information:**

* `in_port`: Receiver port number
* `dl dst`: MAC address of the receiver host
* `port_bytes`: Number of bytes in the flow on the port level
* `port_packet`: Number of packets in the flow on the port level (may be a duplicate of `ip_packet`)
* `port_flow_count`: Number of flows on the port level

**Table Information:**

* `table_active_count`: Number of active entries in the table
* `table_lookup_count`: Number of packets looked up in the table
* `table_matched_count`: Number of packets that matched an entry in the table

**Packet Transfer Information:**

* `port_rx_packets`: Number of received packets on the port level
* `port_tx_packets`: Number of transmitted packets on the port level
* `port_rx_bytes`: Number of received bytes on the port level
* `port_tx_bytes`: Number of transmitted bytes on the port level

**Packet Drop and Error Information:**

* `port_rx_dropped`: Number of packets dropped by the receiver on the port level
* `port_tx_dropped`: Number of packets dropped by the transmitter on the port level
* `port_rx_errors`: Number of received errors
* `port_tx_errors`: Number of transmitted errors

**Frame Error Information:**

* `port_rx_frame_err`: Number of frame alignment errors
* `port_rx_over_err`: Number of received packets with overrun
* `port_rx_crc_err`: Number of CRC errors

**Collision Information:**

* `port_collisions`: Number of collisions

### Data Preprocessing

In [6]:
DATASET_PATH = "../../Dataset/MCAD Dataset/"

# Attack flow data
attack_files = {
    "ddos": [
        "attack_ddos_tcp.csv",
        "attack_ddos_udp.csv",
        "ddos_attack_scapy_new.csv"
    ],
    "probe": [
        "attack_os_port_scan.csv"
    ],
    "web": [
        "attack_sql_injection.csv"
    ],
    "r2l": [
        "attack_bruteforce.csv",
        "attack_cmd.csv"
    ],
    "u2r": [
        "attack_vnc.csv",
        "attack_samba.csv"
    ],
    "normal": [
        "normal_ditg.csv",
        "normal_internet1.csv",
        "normal_internet2.csv",
        "normal_internet3.csv",
        "normal_iperf.csv",
    ]
}

dfs = []

for attack_type, files in attack_files.items():
    for file in files:
        try:
            data = pd.read_csv(DATASET_PATH + file)
            data["type"] = attack_type
            dfs.append(data)
        except FileNotFoundError:
            print(f"Error: File '{file}' not found. Skipping...")


# Concatenate all DataFrames
combined_data = pd.concat(dfs, ignore_index=True)
print("Successfully combined data into a single DataFrame!")

Successfully combined data into a single DataFrame!


In [7]:
# Randomize data, remove previous indices
combined = combined_data.sample(frac=1).reset_index().drop('index', axis = 1)
combined.head()

,src,dst,table_id,ip_bytes,ip_packet,ip_duration,in_port,dl_dst,port_bytes,port_packet,...,port_rx_dropped,port_tx_dropped,port_rx_errors,port_tx_errors,port_rx_frame_err,port_rx_over_err,port_rx_crc_err,port_collisions,port_duration_sec,type
0,10.0.0.4,internet,0,12204,152,2,3,52:75:9b:69:f1:fc,14256,162,...,0,0,0,0,0,0,0,0,1064,normal
1,10.0.0.2,10.0.0.3,0,24750,375,37,2,1a:16:b6:94:29:a4,24750,375,...,0,0,0,0,0,0,0,0,52349,normal
2,192.168.221.129,192.168.18.129,0,206,3,3,1,08:00:27:c8:c8:75,11272,165,...,0,0,0,0,0,0,0,0,47354,probe
3,10.0.0.1,10.0.0.4,0,8470007292,184758,5,1,36:97:e5:0a:a1:9d,8470007292,184758,...,0,0,0,0,0,0,0,0,33443,normal
4,192.168.221.129,192.168.18.129,0,1692,8,3,1,08:00:27:c8:c8:75,15077,45,...,0,0,0,0,0,0,0,0,26325,web


In [8]:
# Extracting relevant features
combined['ip_bytes_sec'] = combined['ip_bytes'] / combined['ip_duration']
combined['ip_packets_sec'] = combined['ip_packet'] / combined['ip_duration']
combined['ip_bytes_packet'] = combined['ip_bytes'] / combined['ip_packet']
combined['port_bytes_sec'] = combined['port_bytes'] / combined['ip_duration']
combined['port_packet_sec'] = combined['port_packet'] / combined['ip_duration']
combined['port_byte_packet'] = combined['port_bytes'] / combined['port_packet']
combined['port_flow_count_sec'] = combined['port_flow_count'] / combined['ip_duration']
combined['table_matched_lookup'] = combined['table_matched_count'] / combined['table_lookup_count']
combined['table_active_lookup'] = combined['table_active_count'] / combined['table_lookup_count']
combined['port_rx_packets_sec'] = combined['port_rx_packets'] / combined['port_duration_sec']
combined['port_tx_packets_sec'] = combined['port_tx_packets'] / combined['port_duration_sec']
combined['port_rx_bytes_sec'] = combined['port_rx_bytes'] / combined['port_duration_sec']
combined['port_tx_bytes_sec'] = combined['port_tx_bytes'] / combined['port_duration_sec']

In [9]:
# Removing identifying features
features = combined[
    [
        'ip_bytes_sec',
        'ip_packets_sec',
        'ip_bytes_packet',
        'port_bytes_sec',
        'port_packet_sec',
        'port_byte_packet',
        'port_flow_count_sec',
        'table_matched_lookup',
        'table_active_lookup',
        'port_rx_packets_sec',
        'port_tx_packets_sec',
        'port_rx_bytes_sec',
        'port_tx_bytes_sec',
        'type'
    ]
].copy()
features.describe()

,ip_bytes_sec,ip_packets_sec,ip_bytes_packet,port_bytes_sec,port_packet_sec,port_byte_packet,port_flow_count_sec,table_matched_lookup,table_active_lookup,port_rx_packets_sec,port_tx_packets_sec,port_rx_bytes_sec,port_tx_bytes_sec
count,5.804040e+05,580404.00,452277.000000,5.974070e+05,5.974070e+05,494550.000000,600639.0,600639.000000,600639.000000,600639.000000,600639.000000,6.006390e+05,6.006390e+05
mean,inf,inf,1202.044452,inf,inf,1041.411101,inf,0.999452,0.000190,2257.680682,2335.643851,6.631135e+06,6.667613e+06
std,NaN,NaN,6414.474897,NaN,NaN,6123.736533,NaN,0.001358,0.001123,10219.996477,11071.044552,2.603833e+07,2.618325e+07
min,0.000000e+00,0.00,42.000000,0.000000e+00,0.000000e+00,42.000000,0.0,0.966710,0.000000,0.075705,0.031126,5.342051e+00,1.945194e+00
25%,6.000000e+01,1.00,69.500000,1.403950e+02,2.004526e+00,72.095238,1.0,0.999298,0.000007,13.510969,2.672675,8.408912e+02,2.456661e+02
50%,5.560000e+02,5.00,81.000000,5.006000e+03,3.600000e+01,103.300000,20.0,0.999996,0.000020,40.713272,7.152422,9.082562e+03,2.374347e+03
75%,8.590550e+05,11400.05,211.500000,1.624788e+06,1.898474e+04,309.021127,87.0,1.000000,0.000149,56.834448,13.714397,2.522739e+04,1.355493e+04
max,inf,inf,59433.840607,inf,inf,59433.840607,inf,1.000000,0.117647,76046.285714,77672.248623,3.613620e+08,3.626903e+08


In [10]:
# Removing inf and NaN values
features.replace([np.inf, -np.inf], np.nan, inplace=True)
features.dropna(inplace=True)
features.describe()

,ip_bytes_sec,ip_packets_sec,ip_bytes_packet,port_bytes_sec,port_packet_sec,port_byte_packet,port_flow_count_sec,table_matched_lookup,table_active_lookup,port_rx_packets_sec,port_tx_packets_sec,port_rx_bytes_sec,port_tx_bytes_sec
count,3.514400e+05,351440.000000,351440.000000,3.514400e+05,351440.000000,351440.000000,351440.000000,351440.000000,351440.000000,351440.000000,351440.000000,3.514400e+05,3.514400e+05
mean,4.654730e+07,3109.895009,1386.777235,4.702906e+07,5180.485586,1297.690580,12.271354,0.999423,0.000080,3747.721088,3907.388170,9.034158e+06,9.101573e+06
std,3.232547e+08,9352.464107,6996.579666,3.258321e+08,15300.665796,6991.416210,25.896976,0.001452,0.000349,13148.260145,14256.084923,2.998490e+07,3.014559e+07
min,1.350000e+01,0.250000,42.000000,1.350000e+01,0.250000,42.000000,0.000070,0.966762,0.000000,0.075705,0.031173,5.342051e+00,1.951717e+00
25%,8.100000e+01,1.001032,69.500000,1.336000e+03,10.105263,70.000000,0.032258,0.999284,0.000005,6.210569,4.372770,5.522635e+02,5.694660e+02
50%,6.648889e+02,5.500000,82.250000,4.883000e+03,34.666667,100.108108,3.500000,0.999995,0.000014,42.562171,7.801970,1.463353e+04,7.416903e+03
75%,7.937619e+03,19.000000,210.444444,7.408800e+04,136.000000,337.571429,16.000000,1.000000,0.000042,58.571233,23.105143,2.682665e+04,1.472640e+04
max,5.466231e+09,102303.000000,57707.308373,5.909109e+09,134900.000000,57707.308373,857.000000,1.000000,0.046875,76046.285714,77672.248623,3.549138e+08,2.893631e+08


In [11]:
# Total attack and normal data
features['type'].value_counts()

type
normal    112688
ddos       72041
u2r        56036
r2l        47482
web        43432
probe      19761
Name: count, dtype: int64

### Train, Test splitting

In [12]:
# Splitting into dependent and independent variables
X = features.loc[ : , features.columns != 'type']
y = features['type']

# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

### Data scaling

In [13]:
# Scaling data using RS
X_train = pd.DataFrame(RobustScaler().fit_transform(X_train), columns=[
    'ip_bytes_sec',
    'ip_packets_sec',
    'ip_bytes_packet',
    'port_bytes_sec',
    'port_packet_sec',
    'port_byte_packet',
    'port_flow_count_sec',
    'table_matched_lookup',
    'table_active_lookup',
    'port_rx_packets_sec',
    'port_tx_packets_sec',
    'port_rx_bytes_sec',
    'port_tx_bytes_sec',
])

X_test = pd.DataFrame(RobustScaler().fit_transform(X_test), columns=[
    'ip_bytes_sec',
    'ip_packets_sec',
    'ip_bytes_packet',
    'port_bytes_sec',
    'port_packet_sec',
    'port_byte_packet',
    'port_flow_count_sec',
    'table_matched_lookup',
    'table_active_lookup',
    'port_rx_packets_sec',
    'port_tx_packets_sec',
    'port_rx_bytes_sec',
    'port_tx_bytes_sec',
])

### Model training and accuracy

Without PCA

In [ ]:
# Fitting model to data
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
# Getting accuracy of the model
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

With PCA

In [14]:
from sklearn.decomposition import PCA

pca = PCA(n_components=6)
X_train2 = pd.DataFrame(pca.fit_transform(X_train))

In [15]:
rf = RandomForestClassifier()
rf.fit(X_train2, y_train)

RandomForestClassifier()

In [16]:
X_test2 = pd.DataFrame(pca.transform(X_test))

In [17]:
y_pred = rf.predict(X_test2)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.957830639653995


**Accuracy records**:
 - Accuracy: 0.9438026405645344
 - Accuracy: 0.9455383564762122
 - Accuracy: 0.9928864101980424